# Capítulo 2: DSPy Essentials & Primeiro Single Agent

**Production-Ready Multi-Agent Systems with DSPy**

---

## Objetivos de Aprendizado

Ao final deste capítulo, você será capaz de:

1. ✅ **Entender core concepts do DSPy:**
   - Signatures (inputs/outputs estruturados)
   - Modules (componentes reutilizáveis)
   - Predictors (wrappers de LLM)
   - ChainOfThought (raciocínio explícito)

2. ✅ **Implementar seu primeiro ReAct agent:**
   - Setup completo do ambiente
   - Data models com Pydantic
   - Tool functions
   - Agent completo funcionando

3. ✅ **Identificar limitações de single agents:**
   - Testar em casos simples (sucesso)
   - Testar em casos complexos (falha)
   - Entender POR QUÊ falha
   - Motivação para multi-agent (Cap 3)

---

## Pré-requisitos

- Python intermediário (classes, type hints)
- Conceitos básicos de LLMs
- Ambiente configurado (ver [Introdução](../../introducao.md))

---

## Tempo Estimado

- **Leitura + Execução:** 60-75 minutos
- **Experimentação:** +30-45 minutos

---

## Estrutura do Capítulo

```
1. Teoria: DSPy Core Concepts
2. Setup e Configuração
3. Data Models (Pydantic)
4. Tool Functions
5. Primeiro Single Agent (ReAct)
6. Testes: Casos Simples (✅ Sucesso)
7. Testes: Casos Complexos (❌ Falha)
8. Análise de Limitações
9. Conclusões e Próximos Passos
```

---

**Vamos começar!** 🚀


## Parte 1: DSPy Core Concepts - Teoria Fundamenta

### O Que É DSPy?

**DSPy** (Declarative Self-improving Language Programs) é um framework criado por Omar Khattab e equipe no Stanford NLP Group que trata **LLM pipelines como programs**, não como strings mágicas.

**Diferença fundamental:**

**Prompt Engineering Tradicional:**
```python
prompt = """
You are a helpful assistant. Given user request, you should analyze it carefully,
use available tools, and provide a detailed response. Always be professional...
[20+ linhas de instruções]
"""
response = llm(prompt + user_input)
```

**Problemas:**
- ❌ Prompts longos e frágeis
- ❌ Difícil otimizar sistematicamente
- ❌ Muito trial-and-error
- ❌ Não reutilizável

**DSPy:**
```python
class MyAgent(dspy.Module):
    def __init__(self):
        self.process = dspy.ChainOfThought(ProcessSignature)
    
    def forward(self, user_input):
        return self.process(user_input=user_input)

# Optimization automática
optimized = optimizer.compile(MyAgent(), trainset=examples)
```

**Vantagens:**
- ✅ Declarativo: define O QUE, não COMO
- ✅ Optimization nativa (MIPRO, BootstrapFewShot)
- ✅ Modular e reutilizável
- ✅ Testável e reproduzível

---

### Core Concepts

DSPy tem 3 conceitos fundamentais:

```
┌─────────────────────────────────────┐
│          DSPy Program               │
│                                     │
│  ┌───────────────────────────────┐ │
│  │   1. SIGNATURE                │ │
│  │   (O QUÊ fazer)               │ │
│  │   inputs → outputs            │ │
│  └───────────────────────────────┘ │
│              ↓                      │
│  ┌───────────────────────────────┐ │
│  │   2. MODULE                   │ │
│  │   (COMO fazer)                │ │
│  │   lógica de processamento     │ │
│  └───────────────────────────────┘ │
│              ↓                      │
│  ┌───────────────────────────────┐ │
│  │   3. PREDICTOR                │ │
│  │   (QUEM executa)              │ │
│  │   wrapper do LLM              │ │
│  └───────────────────────────────┘ │
│                                     │
└─────────────────────────────────────┘
```

Vamos explorar cada um em detalhes.

---

### 1. Signatures: Inputs e Outputs Estruturados

**Signature** define **O QUÊ** seu programa faz: quais inputs recebe e quais outputs produz.

**Conceito:** Similarão a type signatures em linguagens de programação.

```python
# Em Python puro
def soma(a: int, b: int) -> int:
    return a + b
```

**Em DSPy:**
```python
class ProcessRequest(dspy.Signature):
    """Processa requisição do usuário e retorna resposta."""
    
    user_request: str = dspy.InputField(
        desc="Requisição do usuário em linguagem natural"
    )
    response: str = dspy.OutputField(
        desc="Resposta processada para o usuário"
    )
```

**Componentes de uma Signature:**

1. **Docstring:** Descreve o objetivo (usado no prompt)
2. **InputField:** Campos de entrada com descrições
3. **OutputField:** Campos de saída esperados

**Por que Signatures?**
- ✅ Estrutura clara de entrada/saída
- ✅ Type safety
- ✅ Auto-documentação
- ✅ Optimization usa descrições para melhorar prompts

---

### 2. Modules: Componentes Reutilizáveis

**Module** define **COMO** processar: a lógica do seu programa.

**Conceito:** Similar a `nn.Module` no PyTorch, mas para LLMs.

```python
class MyAgent(dspy.Module):
    def __init__(self):
        super().__init__()
        # Inicializa sub-módulos
        self.process = dspy.ChainOfThought(ProcessRequest)
    
    def forward(self, user_request: str) -> dspy.Prediction:
        # Lógica de processamento
        return self.process(user_request=user_request)
```

**Características:**
- Herda de `dspy.Module`
- Implementa `forward()` method
- Pode compor outros modules
- Pode ser otimizado

**Por que Modules?**
- ✅ Reutilizável
- ✅ Testável
- ✅ Composable (modules dentro de modules)
- ✅ Optimization-ready

---

### 3. Predictors: Wrappers do LLM

**Predictor** é **QUEM** executa: o wrapper que chama o LLM.

DSPy oferece vários predictors prontos:

#### `dspy.Predict`

Predictor básico, chamada direta ao LLM.

```python
predict = dspy.Predict(ProcessRequest)
result = predict(user_request="Hello")
```

**Quando usar:** Tasks simples, sem raciocínio complexo.

---

#### `dspy.ChainOfThought`

Adiciona **raciocínio explícito** antes da resposta.

```python
cot = dspy.ChainOfThought(ProcessRequest)
result = cot(user_request="Analyze this contract")
# result.rationale: "Let me think step by step..."
# result.response: "Based on analysis..."
```

**Como funciona:**
1. LLM gera `rationale` (reasoning)
2. Usa rationale para gerar `response`

**Quando usar:** Tasks que beneficiam de reasoning explícito.

**Referência:** Baseado em Chain-of-Thought Prompting [Wei et al., 2022]

---

#### `dspy.ReAct`

**Reasoning + Acting:** alterna entre raciocínio e ações (tool use).

```python
react = dspy.ReAct(ProcessSignature, tools=[search, calculator])
result = react(user_request="What's 2+2 and the weather in SF?")
# Alterna: thought → action (tool) → observation → thought → ...
```

**Como funciona:**
```
1. Thought: "I need to calculate 2+2"
2. Action: calculator(2+2)
3. Observation: 4
4. Thought: "Now I need weather for SF"
5. Action: search_weather("SF")
6. Observation: "Sunny, 72°F"
7. Thought: "I have all information"
8. Answer: "2+2 is 4. Weather in SF is sunny..."
```

**Quando usar:** Tasks que requerem usar tools/APIs externas.

**Referência:** Baseado em ReAct pattern [Yao et al., 2022]

---

### Comparação de Predictors

| Predictor | Raciocínio | Tool Use | Complexidade | Quando Usar |
|-----------|-----------|----------|--------------|-------------|
| `Predict` | ❌ | ❌ | Baixa | Tasks simples, classificação |
| `ChainOfThought` | ✅ | ❌ | Média | Análise, reasoning |
| `ReAct` | ✅ | ✅ | Alta | Agents, tool use |

---

### Exemplo Completo: Anatomia de um DSPy Program

```python
# 1. SIGNATURE: O QUÊ fazer
class AnalyzeText(dspy.Signature):
    """Analisa texto e extrai insights principais."""
    text: str = dspy.InputField(desc="Texto a ser analisado")
    insights: str = dspy.OutputField(desc="Insights extraídos")

# 2. MODULE: COMO fazer
class TextAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        # 3. PREDICTOR: QUEM executa
        self.analyze = dspy.ChainOfThought(AnalyzeText)
    
    def forward(self, text: str):
        return self.analyze(text=text)

# Uso
analyzer = TextAnalyzer()
result = analyzer(text="DSPy é um framework...")
print(result.insights)
```

---

### DSPy vs Outros Frameworks

**vs LangChain:**
- LangChain: orquestração, muitas integrações
- DSPy: optimization nativa, mais structured

**vs Prompt Engineering:**
- Prompt Engineering: manual, trial-and-error
- DSPy: automático, optimization-first

**vs Fine-Tuning:**
- Fine-Tuning: modifica modelo (caro, lento)
- DSPy Optimization: modifica prompts (rápido, barato)

---

### Referências

**Paper Principal:**
> Khattab, O., et al. (2023). **DSPy: Compiling Declarative Language Model Calls into Self-Improving Pipelines.** arXiv:2310.03714.

**Chain-of-Thought:**
> Wei, J., et al. (2022). **Chain-of-Thought Prompting Elicits Reasoning in Large Language Models.** NeurIPS 2022.

**ReAct:**
> Yao, S., et al. (2022). **ReAct: Synergizing Reasoning and Acting in Language Models.** arXiv:2210.03629.

---

**Agora que entendemos a teoria, vamos IMPLEMENTAR nosso primeiro agent!** 💻


## Parte 2: Setup e Configuração

Agora vamos configurar o ambiente para nosso primeiro agent.

### Imports Necessários

Vamos importar tudo que precisamos:


In [ ]:
# Core imports
import dspy
from datetime import datetime, timedelta
from typing import List, Optional, Dict, Any
import json
import uuid
import os

# Data validation
from pydantic import BaseModel, Field

# Environment variables
from dotenv import load_dotenv
load_dotenv()

print("✅ Imports completos!")


### Configuração do LLM

DSPy suporta múltiplos provedores de LLM. Vamos usar **Groq** (rápido e com tier gratuito generoso).

**Alternativas:**
- OpenAI: `dspy.LM(model="openai/gpt-4")`
- Anthropic: `dspy.LM(model="anthropic/claude-3-5-sonnet-20241022")`
- Local: Ollama, vLLM, etc.

**Configuração de API Key:**
```bash
# No terminal ou .env file
export GROQ_API_KEY="your-key-here"
```

**Obter key gratuita:** https://console.groq.com


In [ ]:
# Configurar LLM
lm = dspy.LM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.1  # Baixa temperatura para respostas mais determinísticas
)

# Configurar DSPy para usar este LLM
dspy.configure(lm=lm)

print(f"✅ LLM configurado: {lm.model}")
print(f"   Temperatura: {lm.kwargs.get('temperature', 'default')}")


## Parte 3: Data Models com Pydantic

Vamos modelar nosso domínio: **sistema de booking de voos**.

**Por que Pydantic?**
- ✅ Validação automática de dados
- ✅ Type safety
- ✅ Serialização JSON fácil
- ✅ Integração perfeita com DSPy

### Domain Models

Nosso sistema precisa representar:


In [ ]:
class UserProfile(BaseModel):
    """Perfil do usuário."""
    name: str
    user_id: str
    email: str
    phone: str
    frequent_flyer_number: Optional[str] = None

class Flight(BaseModel):
    """Informações de um voo."""
    flight_id: str
    flight_number: str
    departure_airport: str
    arrival_airport: str
    departure_time: str
    arrival_time: str
    duration_minutes: int
    price: float
    available_seats: int
    
class Itinerary(BaseModel):
    """Itinerário de viagem (pode ter múltiplos voos)."""
    itinerary_id: str
    user_id: str
    flights: List[Flight]
    total_price: float
    booking_date: str
    status: str  # "confirmed", "cancelled", "pending"

print("✅ Models definidos!")
print("\nExemplo de uso:")
user = UserProfile(
    name="Maria",
    user_id="user_001", 
    email="maria@example.com",
    phone="+55-11-98765-4321"
)
print(f"   User: {user.name} ({user.email})")


### Mock Databases

Para simular um sistema real, vamos criar databases em memória:


In [ ]:
# Database de usuários
users_db = {
    "Maria": UserProfile(
        name="Maria",
        user_id="user_001",
        email="maria@example.com",
        phone="+55-11-98765-4321",
        frequent_flyer_number="FF12345"
    ),
    "João": UserProfile(
        name="João",
        user_id="user_002",
        email="joao@example.com",
        phone="+55-11-98765-5678"
    )
}

# Database de voos
flights_db = {
    "GRU-SDU": [  # São Paulo → Rio
        Flight(
            flight_id="f001",
            flight_number="G3-1001",
            departure_airport="GRU",
            arrival_airport="SDU",
            departure_time="08:00",
            arrival_time="09:10",
            duration_minutes=70,
            price=450.00,
            available_seats=15
        ),
        Flight(
            flight_id="f002",
            flight_number="LA-4567",
            departure_airport="GRU",
            arrival_airport="SDU",
            departure_time="14:00",
            arrival_time="15:15",
            duration_minutes=75,
            price=380.00,
            available_seats=8
        )
    ],
    "SDU-GRU": [  # Rio → São Paulo
        Flight(
            flight_id="f003",
            flight_number="G3-1002",
            departure_airport="SDU",
            arrival_airport="GRU",
            departure_time="10:00",
            arrival_time="11:15",
            duration_minutes=75,
            price=420.00,
            available_seats=12
        )
    ]
}

# Databases para bookings (começam vazios)
itineraries_db = {}

print("✅ Databases criados!")
print(f"   Usuários: {len(users_db)}")
print(f"   Rotas: {len(flights_db)}")
print(f"   Total de voos: {sum(len(v) for v in flights_db.values())}")
